# Connect to CARTO database

In this notebook we will learn how to connect to CARTO database using CARTOframes

### Install required libraries

In [1]:
#!pip install -r requirements.txt

In [2]:
from src_import.modules import *
from src_import.func import *

/Users/giulia/anaconda3/lib/python3.7/site-packages/pysal/__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://migrating.pysal.org
  ), VisibleDeprecationWarning)


## Set up credentials

In [3]:
username = 'INSERT YOUR USERNAME HERE'
api_key = 'INSERT YOUR API KEY HERE'
creds = Credentials(username=username, key=api_key)
cc = CartoContext(creds=creds)

## Upload data to your CARTO account

#### Read data from local directory

In [4]:
df_loc = pd.read_csv('./data/UK_Police_street_crimes_2019_04.csv')
df_loc.columns = df_loc.columns.str.replace(' ', '_')
df_loc = df_loc[~((df_loc['Longitude'].isna()) | (df_loc['Latitude'].isna()))]
df_loc.geometry = [Point(xy) for xy in zip(df_loc['Longitude'], df_loc['Latitude'])]
df_loc = df_loc.drop([df_loc.columns[0]], axis=1)
df_loc = GeoDataFrame(df_loc, geometry=df_loc.geometry)

In [5]:
df_loc.Last_outcome_category.unique()

array(['Under investigation', 'Defendant found not guilty',
       'Awaiting court outcome', 'Offender sent to prison',
       'Offender given community sentence',
       'Defendant sent to Crown Court',
       'Offender ordered to pay compensation',
       'Offender given conditional discharge',
       'Offender given suspended prison sentence',
       'Offender given a caution', 'Local resolution',
       'Court case unable to proceed'], dtype=object)

#### Write data to CARTO

In [6]:
cartodb_table = 'uk_police_street_crimes_2019_04'
cc.write(df_loc, table_name = cartodb_table, overwrite = True)

Params: encode_geom, geom_col and everything in kwargs are deprecated and not being used any more
The following columns were changed in the CARTO copy of this dataframe:
Crime_ID -> crime_id
Month -> month
Reported_by -> reported_by
Falls_within -> falls_within
Longitude -> longitude
Latitude -> latitude
Location -> location
LSOA_code -> lsoa_code
LSOA_name -> lsoa_name
Crime_type -> crime_type
Last_outcome_category -> last_outcome_category
Context -> context
Table successfully written to CARTO: https://do-v2-demo.carto.com/dataset/uk_police_street_crimes_2019_04


## Read data from your CARTO account

In [7]:
df_read = cc.read(table_name = cartodb_table)
df_read.head()

,the_geom,crime_id,month,reported_by,falls_within,longitude,latitude,location,lsoa_code,lsoa_name,crime_type,last_outcome_category,context
cartodb_id,,,,,,,,,,,,,
1,0101000020E610000022C66B5ED559C13F46D3D9C9E0CA...,27ba7c00801cbfd96d3843051814f019aede1c6d51a6aa...,2019-04,Metropolitan Police Service,Metropolitan Police Service,0.135554,51.584985,On or near Rose Lane,E01000027,Barking and Dagenham 001A,Burglary,Under investigation,NaN
2,0101000020E6100000C2A6CEA3E2FFC23FD6E07D552ECC...,7566fdb03eb3c4ca3ad3f5ec7b09e0126d5c9f2cd9825d...,2019-04,Metropolitan Police Service,Metropolitan Police Service,0.148434,51.595164,On or near Park/Open Space,E01000027,Barking and Dagenham 001A,Burglary,Under investigation,NaN
3,0101000020E610000090BB0853944BC13FB27F9E060CCA...,cdcce05d8b40d43bd8aa66a23d617a05ab329703599ee8...,2019-04,Metropolitan Police Service,Metropolitan Police Service,0.135119,51.578492,On or near Portland Close,E01000029,Barking and Dagenham 001C,Burglary,Under investigation,NaN
4,0101000020E6100000431D56B8E523C13F0A14B18861C9...,c945cbce31d46db4dd3e38ac3c82084be977f4ee526b80...,2019-04,Metropolitan Police Service,Metropolitan Police Service,0.133908,51.573289,On or near Mayfair Avenue,E01000032,Barking and Dagenham 002B,Burglary,Under investigation,NaN
5,0101000020E61000002E724F57772CC23F46EF54C03DC9...,72520bc03de4189c5b817f3fa673a069b9bdd7cb5c87b4...,2019-04,Metropolitan Police Service,Metropolitan Police Service,0.141982,51.572197,On or near Whalebone Lane North,E01000112,Barking and Dagenham 002E,Burglary,Under investigation,NaN


## Query data from your CARTO account

In [8]:
selected_crime = "'Burglary'"
q = f'''SELECT * FROM {cartodb_table} WHERE crime_type = {selected_crime}'''
q

"SELECT * FROM uk_police_street_crimes_2019_04 WHERE crime_type = 'Burglary'"

In [9]:
df_query = cc.query(q)
df_query.head()

,the_geom,the_geom_webmercator,crime_id,month,reported_by,falls_within,longitude,latitude,location,lsoa_code,lsoa_name,crime_type,last_outcome_category,context
cartodb_id,,,,,,,,,,,,,,
1,0101000020E610000022C66B5ED559C13F46D3D9C9E0CA...,0101000020110F00001CA24AB0E678CD4034B6379FCDA7...,27ba7c00801cbfd96d3843051814f019aede1c6d51a6aa...,2019-04,Metropolitan Police Service,Metropolitan Police Service,0.135554,51.584985,On or near Rose Lane,E01000027,Barking and Dagenham 001A,Burglary,Under investigation,NaN
2,0101000020E6100000C2A6CEA3E2FFC23FD6E07D552ECC...,0101000020110F000046B81A3AE622D0405703E99495A9...,7566fdb03eb3c4ca3ad3f5ec7b09e0126d5c9f2cd9825d...,2019-04,Metropolitan Police Service,Metropolitan Police Service,0.148434,51.595164,On or near Park/Open Space,E01000027,Barking and Dagenham 001A,Burglary,Under investigation,NaN
3,0101000020E610000090BB0853944BC13FB27F9E060CCA...,0101000020110F0000483F5D6BB060CD409B4EABD3AAA6...,cdcce05d8b40d43bd8aa66a23d617a05ab329703599ee8...,2019-04,Metropolitan Police Service,Metropolitan Police Service,0.135119,51.578492,On or near Portland Close,E01000029,Barking and Dagenham 001C,Burglary,Under investigation,NaN
4,0101000020E6100000431D56B8E523C13F0A14B18861C9...,0101000020110F000074BBFC01491DCD40F9C4E7D5C1A5...,c945cbce31d46db4dd3e38ac3c82084be977f4ee526b80...,2019-04,Metropolitan Police Service,Metropolitan Police Service,0.133908,51.573289,On or near Mayfair Avenue,E01000032,Barking and Dagenham 002B,Burglary,Under investigation,NaN
5,0101000020E61000002E724F57772CC23F46EF54C03DC9...,0101000020110F0000EC2CA595AEDECE40374962F090A5...,72520bc03de4189c5b817f3fa673a069b9bdd7cb5c87b4...,2019-04,Metropolitan Police Service,Metropolitan Police Service,0.141982,51.572197,On or near Whalebone Lane North,E01000112,Barking and Dagenham 002E,Burglary,Under investigation,NaN


## Visualize data on a map directly from your CARTO account

In [10]:
ramp = 'opacity(ramp(top($last_outcome_category, 10), bold),.5)'

Map(
    Layer(
        cartodb_table,
        f'''
        color: {ramp}
        width: 3
        strokeColor: transparent 
        ''',
        legend={
            'type': 'color-category',
            'prop': 'color',
            'title': 'Outcome',
            'description': 'UK Police Apr 2019'
        },
        context = cc
    ),
    basemap=basemaps.darkmatter,
    viewport={'zoom': 9.27, 'lat': 51.499978, 'lng': -0.117578},
    show_info=True
)